<a href="https://colab.research.google.com/github/gianninarui/Chatbot/blob/main/Ruidera_MidtermLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
pip install transformers torch nltk flask

In [45]:
import json
from transformers import pipeline

def load_data(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def initialize_pipelines():
    intent_pipeline = pipeline("zero-shot-classification")
    entity_pipeline = pipeline("ner", aggregation_strategy="simple")
    return intent_pipeline, entity_pipeline

def determine_intent(intent_pipeline, user_message, candidate_labels):
    result = intent_pipeline(user_message, candidate_labels=candidate_labels)
    return result

def identify_entities(entity_pipeline, user_message):
    extracted_entities = entity_pipeline(user_message)
    return extracted_entities

def generate_chatbot_response(intent_pipeline, entity_pipeline, data, user_message):
    candidate_labels = [entry['intent'] for entry in data]

    intent_outcome = determine_intent(intent_pipeline, user_message, candidate_labels)
    entities_outcome = identify_entities(entity_pipeline, user_message)

    top_intent = intent_outcome['labels'][0]
    top_score = intent_outcome['scores'][0]

    formatted_entities = [
        {
            "word": entity['word'],
            "score": entity['score'],
            "start": entity['start'],
            "end": entity['end']
        } for entity in entities_outcome
    ]

    response = f"Intent: {top_intent} (Score: {top_score:.2f})\nEntities: "
    for entity in formatted_entities:
        response += f"{entity['word']} (Score: {entity['score']:.2f}, Start: {entity['start']}, End: {entity['end']}) "

    return response

def collect_rating():
    while True:
        try:
            user_rating = int(input("Rate the chatbot response from 1 to 5: "))
            if 1 <= user_rating <= 5:
                if user_rating == 1 or user_rating == 2:
                    feedback = "Unsatisfactory rating. Please improve the model."
                elif user_rating == 3:
                    feedback = "Fair rating. Please explore more models."
                elif user_rating in (4, 5):
                    feedback = "Satisfactory rating. Keep up the good work!"

                print(f"\nRating: {user_rating} - {feedback}")
                break
            else:
                print("Please enter a rating between 1 and 5.")
        except ValueError:
            print("Invalid input. Please enter a number between 1 and 5.")

def execute_chatbot():
    data_file = 'dataset.json'
    print(f"Loading dataset from {data_file}...")
    data = load_data(data_file)

    intent_pipeline, entity_pipeline = initialize_pipelines()

    user_message = "Find me a hotel in Paris."

    response = generate_chatbot_response(intent_pipeline, entity_pipeline, data, user_message)
    print(f"\nChatbot: \n{response}\n")

    collect_rating()

if __name__ == "__main__":
    execute_chatbot()

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


Loading dataset from dataset.json...


No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Chatbot: 
Intent: find_place (Score: 0.08)
Entities: Paris (Score: 1.00, Start: 19, End: 24) 

Rate the chatbot response from 1 to 5: 5

Rating: 5 - Satisfactory rating. Keep up the good work!
